<a href="https://colab.research.google.com/github/akashe/Python-Code-Generation/blob/main/Conala_with_original_data_with_python_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy import data
from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time
import pickle
import json

In [3]:
! git clone https://github.com/akashe/Python-Code-Generation

Cloning into 'Python-Code-Generation'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 82 (delta 39), reused 40 (delta 10), pack-reused 0
Unpacking objects: 100% (82/82), done.


In [4]:
import sys
sys.path.append("/content/Python-Code-Generation/")

In [5]:
from data_processing import getTokenizer,getData

In [6]:
!wget -c "http://www.phontron.com/download/conala-corpus-v1.1.zip"

--2021-03-12 15:03:49--  http://www.phontron.com/download/conala-corpus-v1.1.zip
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52105440 (50M) [application/zip]
Saving to: ‘conala-corpus-v1.1.zip’

conala-corpus-v1.1. 100%[===================>]  49.69M  30.4MB/s    in 1.6s    

2021-03-12 15:03:51 (30.4 MB/s) - ‘conala-corpus-v1.1.zip’ saved [52105440/52105440]



In [7]:
!unzip -n conala-corpus-v1.1.zip

Archive:  conala-corpus-v1.1.zip
   creating: conala-corpus/
  inflating: conala-corpus/conala-mined.jsonl  
  inflating: conala-corpus/conala-train.json  
  inflating: conala-corpus/conala-test.json  


In [8]:
!head -20 conala-corpus/conala-test.json

[
  {
    "intent": "How can I send a signal from a python program?",
    "rewritten_intent": "send a signal `signal.SIGUSR1` to the current process",
    "snippet": "os.kill(os.getpid(), signal.SIGUSR1)",
    "question_id": 15080500
  },
  {
    "intent": "Decode Hex String in Python 3",
    "rewritten_intent": "decode a hex string '4a4b4c' to UTF-8.",
    "snippet": "bytes.fromhex('4a4b4c').decode('utf-8')",
    "question_id": 3283984
  },
  {
    "intent": "check if all elements in a list are identical",
    "rewritten_intent": "check if all elements in list `myList` are identical",
    "snippet": "all(x == myList[0] for x in myList)",
    "question_id": 3844801
  },
  {


In [9]:
questions, answers = [],[]

In [10]:
# loading train json
f = open("/content/conala-corpus/conala-train.json","r")
train_file = json.load(f)
f.close()

In [11]:
for num,i in enumerate(train_file):
  if i['intent'] is not None:
    questions.append(i['intent'])
    answers.append(i['snippet'])

In [12]:
# loading train json
f = open("/content/conala-corpus/conala-test.json","r")
test_file = json.load(f)
f.close()

In [13]:
for num,i in enumerate(test_file):
  if i['intent'] is not None:
    questions.append(i['intent'])
    answers.append(i['snippet'])


In [14]:
questions_, answers_ = getData("/content/Python-Code-Generation/data/english_python_data_pruned.txt")

In [15]:
# Setting max word len
max_word_len = 301

In [16]:
# removing examples with len more than max_word_len
# Only checking in original data because I checked in CONALA data and answers their dont exceed 301.
pruned_questions = []
pruned_answers = []
for j,i in zip(questions_,answers_):
  tokens = getTokenizer(i)
  if not len(tokens) > max_word_len:
    pruned_answers.append(i)
    pruned_questions.append(j)

print(len(pruned_answers))
answers_ = pruned_answers
questions_ = pruned_questions

Error in tokenization
3058


In [17]:
questions = questions + questions_
answers = answers + answers_

In [18]:
questions[100],answers[100]

('Clicking a link using selenium using python',
 "driver.find_element_by_xpath('xpath').click()")

In [19]:
print(len(questions),len(answers))

5937 5937


In [20]:
SEED = 1327

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [21]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [22]:
spacy_en = spacy.load('en')

In [23]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [24]:
SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = getTokenizer, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = False, 
            batch_first = True)

In [25]:
fields = [('src', SRC), ('trg', TRG)]

Examples = [data.Example.fromlist([i,j], fields) for i,j in zip(questions,answers)]
Dataset = data.Dataset(Examples, fields)

Error in tokenization


In [26]:
train_data,valid_data = Dataset.split(split_ratio=[0.85,0.15])

In [27]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [28]:
print(len(SRC.vocab))
print(len(TRG.vocab))

1661
2666


In [29]:
# Dumps dicts
with open("/content/SRC_stio_local","wb") as f:
  pickle.dump(SRC.vocab.stoi,f)
with open("/content/TRG_itos_local","wb") as f:
  pickle.dump(TRG.vocab.itos,f)

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [31]:
BATCH_SIZE = 32

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
     batch_size = BATCH_SIZE,
     sort_key = lambda x: len(x.trg),
     device = device)

In [32]:
class PositionalEncodingComponent(nn.Module):
  '''
  Class to encode positional information to tokens.
  

  '''
  def __init__(self,hid_dim,device,dropout=0.2,max_len=5000):
    super().__init__()

    assert hid_dim%2==0 # If not, it will result error in allocation to positional_encodings[:,1::2] later

    self.dropout = nn.Dropout(dropout)

    self.positional_encodings = torch.zeros(max_len,hid_dim)

    pos = torch.arange(0,max_len).unsqueeze(1) # pos : [max_len,1]
    div_term  = torch.exp(-torch.arange(0,hid_dim,2)*math.log(10000.0)/hid_dim) # Calculating value of 1/(10000^(2i/hid_dim)) in log space and then exponentiating it
    # div_term: [hid_dim//2]

    self.positional_encodings[:,0::2] = torch.sin(pos*div_term) # pos*div_term [max_len,hid_dim//2]
    self.positional_encodings[:,1::2] = torch.cos(pos*div_term) 

    self.positional_encodings = self.positional_encodings.unsqueeze(0) # To account for batch_size in inputs

    self.device = device

  def forward(self,x):
    x = x + self.positional_encodings[:,:x.size(1)].detach().to(self.device)
    return self.dropout(x)

In [33]:
class FeedForwardComponent(nn.Module):
  '''
  Class for pointwise feed forward connections
  '''
  def __init__(self,hid_dim,pf_dim,dropout):
    super().__init__()

    self.dropout = nn.Dropout(dropout)

    self.fc1 = nn.Linear(hid_dim,pf_dim)
    self.fc2 = nn.Linear(pf_dim,hid_dim)

  def forward(self,x):

    # x : [batch_size,seq_len,hid_dim]
    x = self.dropout(torch.relu(self.fc1(x)))

    # x : [batch_size,seq_len,pf_dim]
    x = self.fc2(x)

    # x : [batch_size,seq_len,hid_dim]
    return x

In [34]:
class MultiHeadedAttentionComponent(nn.Module):
  '''
  Multiheaded attention Component. This implementation also supports mask. 
  The reason for mask that in Decoder, we don't want attention mechanism to get
  important information from future tokens.
  '''
  def __init__(self,hid_dim, n_heads, dropout, device):
    super().__init__()

    assert hid_dim % n_heads == 0 # Since we split hid_dims into n_heads

    self.hid_dim = hid_dim
    self.n_heads = n_heads # no of heads in 'multiheaded' attention
    self.head_dim = hid_dim//n_heads # dims of each head

    # Transformation from source vector to query vector
    self.fc_q = nn.Linear(hid_dim,hid_dim)

    # Transformation from source vector to key vector
    self.fc_k = nn.Linear(hid_dim,hid_dim)

    # Transformation from source vector to value vector
    self.fc_v = nn.Linear(hid_dim,hid_dim)

    self.fc_o = nn.Linear(hid_dim,hid_dim)

    self.dropout = nn.Dropout(dropout)

    # Used in self attention for smoother gradients
    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self,query,key,value,mask=None):

    #query : [batch_size, query_len, hid_dim]
    #key : [batch_size, key_len, hid_dim]
    #value : [batch_size, value_len, hid_dim]

    batch_size = query.shape[0]

    # Transforming quey,key,values
    Q = self.fc_q(query)
    K = self.fc_k(key)
    V = self.fc_v(value)

    #Q : [batch_size, query_len, hid_dim]
    #K : [batch_size, key_len, hid_dim]
    #V : [batch_size, value_len,hid_dim]

    # Changing shapes to acocmadate n_heads information
    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

    #Q : [batch_size, n_heads, query_len, head_dim]
    #K : [batch_size, n_heads, key_len, head_dim]
    #V : [batch_size, n_heads, value_len, head_dim]

    # Calculating alpha
    score = torch.matmul(Q,K.permute(0,1,3,2))/self.scale
    # score : [batch_size, n_heads, query_len, key_len]

    if mask is not None:
      score = score.masked_fill(mask==0,-1e10)

    alpha = torch.softmax(score,dim=-1)
    # alpha : [batch_size, n_heads, query_len, key_len]

    # Get the final self-attention  vector
    x = torch.matmul(self.dropout(alpha),V)
    # x : [batch_size, n_heads, query_len, head_dim]

    # Reshaping self attention vector to concatenate
    x = x.permute(0,2,1,3).contiguous()
    # x : [batch_size, query_len, n_heads, head_dim]

    x = x.view(batch_size,-1,self.hid_dim)
    # x: [batch_size, query_len, hid_dim]

    # Transforming concatenated outputs 
    x = self.fc_o(x)
    #x : [batch_size, query_len, hid_dim] 

    return x, alpha

In [35]:
class EncoderLayer(nn.Module):  
  '''
  Operations of a single layer in an Encoder. An Encoder employs multiple such layers. Each layer contains:
  1) multihead attention, folllowed by
  2) LayerNorm of addition of multihead attention output and input to the layer, followed by
  3) FeedForward connections, followed by
  4) LayerNorm of addition of FeedForward outputs and output of previous layerNorm.
  '''
  def __init__(self, hid_dim,n_heads,pf_dim,dropout,device):
    super().__init__()
    
    self.self_attn_layer_norm = nn. LayerNorm(hid_dim) #Layer norm after self-attention
    self.ff_layer_norm = nn.LayerNorm(hid_dim) # Layer norm after FeedForward component

    self.self_attention = MultiHeadedAttentionComponent(hid_dim,n_heads,dropout,device)
    self.feed_forward = FeedForwardComponent(hid_dim,pf_dim,dropout)

    self.dropout = nn.Dropout(dropout)
    
  def forward(self,src,src_mask):
    
    # src : [batch_size, src_len, hid_dim]
    # src_mask : [batch_size, 1, 1, src_len]

    # get self-attention
    _src, _ = self.self_attention(src,src,src,src_mask)

    # LayerNorm after dropout
    src = self.self_attn_layer_norm(src + self.dropout(_src))
    # src : [batch_size, src_len, hid_dim]

    # FeedForward
    _src = self.feed_forward(src)

    # layerNorm after dropout
    src = self.ff_layer_norm(src + self.dropout(_src))
    # src: [batch_size, src_len, hid_dim]

    return src

In [36]:
class DecoderLayer(nn.Module):
  '''
  Operations of a single layer in an Decoder. An Decoder employs multiple such layers. Each layer contains:
  1) masked decoder self attention, followed by
  2) LayerNorm of addition of previous attention output and input to the layer,, followed by
  3) encoder self attention, followed by
  4) LayerNorm of addition of result of encoder self attention and its input, followed by
  5) FeedForward connections, followed by
  6) LayerNorm of addition of Feedforward results and its input.
  '''
  def __init__(self,hid_dim,n_heads,pf_dim,dropout,device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
    self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
    self.ff_layer_norm = nn.LayerNorm(hid_dim)

    # decoder self attention
    self.self_attention = MultiHeadedAttentionComponent(hid_dim,n_heads,dropout,device)

    # encoder attention
    self.encoder_attention = MultiHeadedAttentionComponent(hid_dim,n_heads,dropout,device)

    # FeedForward
    self.feed_forward = FeedForwardComponent(hid_dim,pf_dim,dropout)

    self.dropout = nn.Dropout(dropout)

  def forward(self,trg, enc_src,trg_mask,src_mask):

    #trg : [batch_size, trg_len, hid_dim]
    #enc_src : [batch_size, src_len, hid_dim]
    #trg_mask : [batch_size, 1, trg_len, trg_len]
    #src_mask : [batch_size, 1, 1, src_len]

    '''
    Decoder self-attention
    trg_mask is to force decoder to look only into past tokens and not get information from future tokens.
    Since we apply mask before doing softmax, the final self attention vector gets no information from future tokens.
    '''
    _trg, _ = self.self_attention(trg,trg,trg,trg_mask)

    # LayerNorm and dropout with resdiual connection
    trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
    # trg : [batch_size, trg_len, hid_dim]

    '''
    Encoder attention:
    Query: trg
    key: enc_src
    Value : enc_src
    Why? 
    the idea here is to extract information from encoder outputs. So we use decoder self-attention as a query to find important values from enc_src
    and that is why we use src_mask, to avoid getting information from enc_src positions where it is equal to pad-id
    After we get necessary infromation from encoder outputs we add them back to decoder self-attention.
    '''
    _trg, encoder_attn_alpha = self.encoder_attention(trg,enc_src,enc_src,src_mask)

        # LayerNorm , residual connection and dropout
    trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
    # trg : [ batch_size, trg_len, hid_dim]

    # Feed Forward
    _trg = self.feed_forward(trg)

    # LayerNorm, residual connection and dropout
    trg = self.ff_layer_norm(trg + self.dropout(_trg))

    return trg, encoder_attn_alpha

In [37]:
class Encoder(nn.Module):
  '''
  An encoder, creates token embeddings and position embeddings and passes them through multiple encoder layers
  '''
  def __init__(self,input_dim,hid_dim,n_layers,n_heads,pf_dim,dropout,device,max_length = 5000):
    super().__init__()
    self.device = device

    self.tok_embedding = nn.Embedding(input_dim,hid_dim)
    self.pos_embedding = PositionalEncodingComponent(hid_dim,device,dropout,max_length)

    # encoder layers
    self.layers = nn.ModuleList([EncoderLayer(hid_dim,n_heads,pf_dim,dropout,device) for _ in range(n_layers)])

    self.dropout = nn.Dropout(dropout)

    self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

  def forward(self,src,src_mask):

    # src : [batch_size, src_len]
    # src_mask : [batch_size,1,1,src_len]

    batch_size = src.shape[0]
    src_len = src.shape[1]

    tok_embeddings = self.tok_embedding(src)*self.scale

    # token plus position embeddings
    src  = self.pos_embedding(tok_embeddings)

    for layer in self.layers:
      src = layer(src,src_mask)
    # src : [batch_size, src_len, hid_dim]

    return src

In [38]:
class Decoder(nn.Module):
  '''
  An decoder, creates token embeddings and position embeddings and passes them through multiple decoder layers
  '''
  def __init__(self,output_dim,hid_dim,n_layers,n_heads,pf_dim,dropout,device,max_length= 5000):
    super().__init__()

    self.device = device

    self.tok_embedding = nn.Embedding(output_dim,hid_dim)
    self.pos_embedding = PositionalEncodingComponent(hid_dim,device,dropout,max_length)

    # decoder layers
    self.layers = nn.ModuleList([DecoderLayer(hid_dim,n_heads,pf_dim,dropout,device) for _ in range(n_layers)])

    # convert decoder outputs to real outputs
    self.fc_out = nn.Linear(hid_dim,output_dim)

    self.dropout = nn.Dropout(dropout)

    self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

  def forward(self, trg, enc_src,trg_mask,src_mask):
    
    #trg : [batch_size, trg_len]
    #enc_src : [batch_size, src_len, hid_dim]
    #trg_mask : [batch_size, 1, trg_len, trg_len]
    #src_mask : [batch_size, 1, 1, src_len]

    batch_size = trg.shape[0]
    trg_len = trg.shape[1]

    tok_embeddings = self.tok_embedding(trg)*self.scale

    # token plus pos embeddings
    trg = self.pos_embedding(tok_embeddings)
    # trg : [batch_size, trg_len, hid_dim]

    # Pass trg thorugh decoder layers
    for layer in self.layers:
      trg, encoder_attention = layer(trg,enc_src,trg_mask,src_mask)
    
    # trg : [batch_size,trg_len,hid_dim]
    # encoder_attention :  [batch_size, n_head,trg_len, src_len]

    # Convert to outputs
    output = self.fc_out(trg)
    # output : [batch_size, trg_len, output_dim]
    
    return output, encoder_attention

In [39]:

class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx
    self.device = device

  def make_src_mask(self,src):
    # src : [batch_size, src_len]

    # Masking pad values
    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    # src_mask : [batch_size,1,1,src_len]

    return src_mask

  def make_trg_mask(self,trg):
    # trg : [batch_size, trg_len]

    # Masking pad values
    trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
    # trg_pad_mask : [batch_size,1,1, trg_len]

    # Masking future values
    trg_len = trg.shape[1]
    trg_sub_mask = torch.tril(torch.ones((trg_len,trg_len),device= self.device)).bool()
    # trg_sub_mask : [trg_len, trg_len]

    # combine both masks
    trg_mask = trg_pad_mask & trg_sub_mask
    # trg_mask = [batch_size,1,trg_len,trg_len]

    return trg_mask

  def forward(self,src,trg):

    # src : [batch_size, src_len]
    # trg : [batch_size, trg_len]

    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)

    # src_mask : [ batch_size, 1,1,src_len]
    # trg_mask : [batch_size, 1, trg_len, trg_len]

    enc_src = self.encoder(src,src_mask)
    #enc_src : [batch_size, src_len, hid_dim]

    output, encoder_decoder_attention = self.decoder(trg,enc_src,trg_mask,src_mask)
    # output : [batch_size, trg_len, output_dim]
    # encoder_decoder_attention : [batch_size, n_heads, trg_len, src_len]

    return output, encoder_decoder_attention

In [40]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 1024
DEC_PF_DIM = 1024
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [41]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights);

In [42]:
model.decoder.tok_embedding

Embedding(2666, 256)

In [44]:
# load python embedding weights
pretrained_embeddings = torch.load('python_embedding_weigts.pt')
pretrained_embeddings

Embedding(15018, 256)

In [53]:
# Loading pretrained embeddings

with open("SRC_stio","rb") as f:
  stoi_weights = pickle.load(f)

with torch.no_grad():
  indexes = []
  for i,j in enumerate(TRG.vocab.stoi):
    if j in stoi_weights:
      model.decoder.tok_embedding.weight[TRG.vocab.stoi[j]] = pretrained_embeddings.weight[stoi_weights[j]]
    else:
      model.decoder.tok_embedding.weight[TRG.vocab.stoi[j]] = pretrained_embeddings.weight[stoi_weights['<unk>']]



In [55]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,322,474 trainable parameters


In [57]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [58]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [59]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [60]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [61]:
N_EPOCHS = 40
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'conala_plus_original_data.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 15s
	Train Loss: 3.800 | Train PPL:  44.711
	 Val. Loss: 2.664 |  Val. PPL:  14.355
Epoch: 02 | Time: 0m 15s
	Train Loss: 2.624 | Train PPL:  13.786
	 Val. Loss: 2.263 |  Val. PPL:   9.611
Epoch: 03 | Time: 0m 16s
	Train Loss: 2.299 | Train PPL:   9.961
	 Val. Loss: 2.077 |  Val. PPL:   7.980
Epoch: 04 | Time: 0m 16s
	Train Loss: 2.089 | Train PPL:   8.077
	 Val. Loss: 1.925 |  Val. PPL:   6.856
Epoch: 05 | Time: 0m 16s
	Train Loss: 1.929 | Train PPL:   6.880
	 Val. Loss: 1.849 |  Val. PPL:   6.356
Epoch: 06 | Time: 0m 16s
	Train Loss: 1.795 | Train PPL:   6.020
	 Val. Loss: 1.780 |  Val. PPL:   5.928
Epoch: 07 | Time: 0m 16s
	Train Loss: 1.681 | Train PPL:   5.372
	 Val. Loss: 1.722 |  Val. PPL:   5.596
Epoch: 08 | Time: 0m 16s
	Train Loss: 1.571 | Train PPL:   4.812
	 Val. Loss: 1.702 |  Val. PPL:   5.486
Epoch: 09 | Time: 0m 16s
	Train Loss: 1.486 | Train PPL:   4.419
	 Val. Loss: 1.669 |  Val. PPL:   5.304
Epoch: 10 | Time: 0m 16s
	Train Loss: 1.404 | Train PPL

In [62]:

import pickle

with open("SRC_stio_local","rb") as f:
  stoi = pickle.load(f)
with open("TRG_itos_local","rb") as f:
  itos = pickle.load(f)

# Load model
trained_model = 'conala_plus_original_data.pt'
model.load_state_dict(torch.load(trained_model));
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (tok_embedding): Embedding(1661, 256)
    (pos_embedding): PositionalEncodingComponent(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadedAttentionComponent(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): FeedForwardComponent(
          (dropout): Dropout(p=0.1, inplace=False)
          (fc1): Linear(in_features=256, out_features=1024, bias=True)
          (fc2): Linear(in_features=10

In [63]:
import spacy
spacy_en = spacy.load('en')

def encode_inputs(input,vocab):

  tokenized_input = [tok.text.lower() for tok in spacy_en.tokenizer(input)]
  tokenized_input = ['<sos>'] + tokenized_input +['<eos>']

  numericalized_input = [vocab[i] for i in tokenized_input]

  tensor_input = torch.LongTensor([numericalized_input])
  
  return tensor_input

def decode_outputs(output,vocab):
  # output: [1,1,hid_dim]
  predicted_token = output.argmax(-1)
  return vocab[predicted_token.item()], predicted_token

In [65]:
 print(" Enter q or quit to exit.")

answer_max_len = 200

while(True):

  input_ = input("Enter text:")

  if input_=='q' or input_=='quit':
    break

  src = encode_inputs(input_,stoi).to(device)
  # src_mask = torch.ones([1,1,1,src.shape[-1]]).to(device)
  src_mask = model.make_src_mask(src)

  with torch.no_grad():
    enc_src = model.encoder(src,src_mask)
  
  trg = '<sos>'
  trg_indexes = [stoi[trg]]
  # trg_mask = torch.ones([1,1,1,1]).to(device)

  decoder_outputs = []
  for i in range(answer_max_len):
    trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
    trg_mask = model.make_trg_mask(trg_tensor)
    
    with torch.no_grad():
      decoder_output,_ = model.decoder(trg_tensor,enc_src,trg_mask,src_mask)

    pred_token = decoder_output.argmax(2)[:,-1].item()

    if pred_token == TRG.vocab.stoi[TRG.eos_token]:
      break
    decoder_outputs.append(itos[pred_token])
    trg_indexes.append(pred_token)

    
  print(" ".join(decoder_outputs))

 Enter q or quit to exit.
Enter text:write a program to print even length words in a string
sentence = ' t h e   q u i c k   b r o w n   f o x ' 
 words = sentence . split ( '   ' ) 
 words . split ( ) 
 print ( '   ' . join ( words ) ) 
Enter text:write a program to accept the strings which contains all vowels
vowels = ' a e i o u ' 
 ip_str = ' a e i o u ' 
 ip_str = ip_str . casefold ( ) 
 count = { } . fromkeys ( vowels , ' a ' a ' ) 
 count = { } . fromkeys ( vowels , 0 ) 
 for char in ip_str : 
 	 if char in count : 
 		 count [ char ] += 1 
   print ( count ) 
Enter text:q
